# Data Cleaning - AI Salary Analysis: Remote Work and Experience Premiums

## Purpose

This notebook prepares the raw AI salary data for analysis by:
1. Filtering to the US market and full-time positions 
2. Creating clean remote work classifications 
3. Categorizing job roles for hypothesis testing 
4. Removing outliers and selecting releavnt variables 

**input:** 'salaries.csv'

**Output:** 'cleaned_salaries.csv'

## Load Raw Data and Import Libraries

We will import out nessesary packages and load our raw data into this notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


df_raw = pd.read_csv('salaries.csv')

print(f"Raw data loaded: {df_raw.shape[0]:,} rows, {df_raw.shape[1]} columns")
print(f"\nColumns: {df_raw.columns.tolist()}")
print(f"\nFirst 5 rows:")
df_raw.head()
df_raw.info()


Raw data loaded: 151,445 rows, 11 columns

Columns: ['work_year', 'experience_level', 'employment_type', 'job_title', 'salary', 'salary_currency', 'salary_in_usd', 'employee_residence', 'remote_ratio', 'company_location', 'company_size']

First 5 rows:


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,EX,FT,Head of Data,348516,USD,348516,US,0,US,M
1,2025,EX,FT,Head of Data,232344,USD,232344,US,0,US,M
2,2025,SE,FT,Data Scientist,145400,USD,145400,US,0,US,M
3,2025,SE,FT,Data Scientist,81600,USD,81600,US,0,US,M
4,2025,MI,FT,Engineer,160000,USD,160000,US,100,US,M


To give context to the data, I printed the raw info for the data and also investigated if there were any NaN values within the datatset

In [3]:
print(df_raw.info())

print(df_raw.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151445 entries, 0 to 151444
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   work_year           151445 non-null  int64 
 1   experience_level    151445 non-null  object
 2   employment_type     151445 non-null  object
 3   job_title           151445 non-null  object
 4   salary              151445 non-null  int64 
 5   salary_currency     151445 non-null  object
 6   salary_in_usd       151445 non-null  int64 
 7   employee_residence  151445 non-null  object
 8   remote_ratio        151445 non-null  int64 
 9   company_location    151445 non-null  object
 10  company_size        151445 non-null  object
dtypes: int64(4), object(7)
memory usage: 12.7+ MB
None
work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remo

## Filter to US Employees Only

To align with my research question, we will filter to only US Employees. This will make it easier for me to look through my data and remove data that is not useful for my research question


In [4]:
print(df_raw['employee_residence'].value_counts().head(10))
print(f"\nUS employees: {(df_raw['employee_residence'] == 'US').sum():,} ({(df_raw['employee_residence'] == 'US').sum() / len(df_raw) * 100:.1f}%)")

# Filter to US only
df = df_raw[df_raw['employee_residence'] == 'US'].copy()

print(f"\n=== FILTERING RESULT ===")
print(f"Rows before: {len(df_raw):,}")
print(f"Rows after: {len(df):,}")
print(f"Rows removed: {len(df_raw) - len(df):,}")

employee_residence
US    135506
CA      6207
GB      4172
AU       590
NL       482
DE       445
FR       441
LT       372
AT       355
ES       313
Name: count, dtype: int64

US employees: 135,506 (89.5%)

=== FILTERING RESULT ===
Rows before: 151,445
Rows after: 135,506
Rows removed: 15,939


## Filter to Full-Time Positions Only

I will filter for only full-time positions because part-time and contract positions tend to have fundamentlaly different compensation structures. It is most releavnt to my research question that we only see the patterns of salary in full-time positions

In [5]:
print(df['employment_type'].value_counts())
print(f"\nFull-time percentage: {(df['employment_type'] == 'FT').sum() / len(df) * 100:.2f}%")

# Filter to full-time only
df = df[df['employment_type'] == 'FT'].copy()

print(f"\n=== FILTERING RESULT ===")
print(f"Keeping only FT (Full-Time) positions")
print(f"Rows after filtering: {len(df):,}")
print(f"Removed: {(df_raw['employee_residence'] == 'US').sum() - len(df):,} non-FT positions")

employment_type
FT    134874
CT       317
PT       315
Name: count, dtype: int64

Full-time percentage: 99.53%

=== FILTERING RESULT ===
Keeping only FT (Full-Time) positions
Rows after filtering: 134,874
Removed: 632 non-FT positions


## Create Binary Remote Work Variable

This dataset uses 'remote_ratio' with three values:

- 0 = Fully on-site
- 50 = Hybrid
- 100 = Fully remote 

I will remove the number of hybrid workers, because I want a clean comparison between fully on-site and fully remote positions. This helps me create a clear binary classsification that is relevant to my research question. 

In [6]:
print("Remote ratio distribution:")
print(df['remote_ratio'].value_counts().sort_index())

df = df[df['remote_ratio'] != 50].copy()
df['remote_work'] = (df['remote_ratio'] == 100).astype(int)

print(f"\nAfter removing hybrid workers: {len(df):,} rows")
print("\nRemote work breakdown:")
print(f"  On-site: {(df['remote_work']==0).sum():,} ({(df['remote_work']==0).sum()/len(df)*100:.1f}%)")
print(f"  Remote:  {(df['remote_work']==1).sum():,} ({(df['remote_work']==1).sum()/len(df)*100:.1f}%)")

Remote ratio distribution:
remote_ratio
0      106647
50         49
100     28178
Name: count, dtype: int64

After removing hybrid workers: 134,825 rows

Remote work breakdown:
  On-site: 106,647 (79.1%)
  Remote:  28,178 (20.9%)


## Categorize Job Titles into Specialized vs Generalist Roles

I will categorize based on a basic understanding of the depth of specialized AI/ML knowledege required for each role and a survey through the dataset.

**Specialized roles** (deep ML/AI technical expertise):
- Machine Learning Engineer, Scientist, Architect
- Research Scientist  
- AI Engineer
- Computer Vision Engineer
- Research Engineer
- Applied Scientist
- NLP Engineer
- Deep Learning Engineer

**Generalist roles** (broader data/analytics work):
- Data Scientist
- Data Analyst
- Data Engineer
- Business Intelligence roles
- Analytics Engineer
- All other positions

In [7]:
specialized_keywords = [
    'Machine Learning',
    'Research Scientist',
    'AI Engineer',
    'Computer Vision',
    'Research Engineer',
    'Applied Scientist',
    'NLP',
    'Deep Learning'
]

def categorize_role(job_title):
    """Categorize job title as Specialized or Generalist based on ML/AI keywords."""
    job_title_upper = str(job_title).upper()
    for keyword in specialized_keywords:
        if keyword.upper() in job_title_upper:
            return 'Specialized'
    return 'Generalist'

df['role_type'] = df['job_title'].apply(categorize_role)

print("Role type distribution:")
print(df['role_type'].value_counts())
print(f"\nTop 5 specialized roles:")
print(df[df['role_type'] == 'Specialized']['job_title'].value_counts().head(5))
print(f"\nTop 5 generalist roles:")
print(df[df['role_type'] == 'Generalist']['job_title'].value_counts().head(5))

Role type distribution:
role_type
Generalist     116210
Specialized     18615
Name: count, dtype: int64

Top 5 specialized roles:
job_title
Machine Learning Engineer    8082
Research Scientist           3310
Applied Scientist            2313
AI Engineer                  1748
Research Engineer            1621
Name: count, dtype: int64

Top 5 generalist roles:
job_title
Data Scientist       16874
Software Engineer    15818
Data Engineer        14220
Data Analyst         11633
Engineer             10234
Name: count, dtype: int64


## Map Experience Levels to Readable Names

I will convert experience level codes (EN, MI, SE, EX) to readbale labels for eas of readability reasons. 

In [8]:
experience_map = {
    'EN': 'Entry',
    'MI': 'Mid',
    'SE': 'Senior',
    'EX': 'Executive'
}

df['experience_level'] = df['experience_level'].map(experience_map)

print("Experience level distribution:")
print(df['experience_level'].value_counts())

Experience level distribution:
experience_level
Senior       79940
Mid          40289
Entry        10768
Executive     3828
Name: count, dtype: int64


## Remove Salary Outliers

We will use the Interquartile Range to detect any outliers in salaries. This will hopefully remove any data pullers or errors to preserve legitimate high earners

In [9]:
Q1 = df['salary_in_usd'].quantile(0.25)
Q3 = df['salary_in_usd'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1 (25th percentile): ${Q1:,.2f}")
print(f"Q3 (75th percentile): ${Q3:,.2f}")
print(f"IQR: ${IQR:,.2f}")
print(f"Outlier boundaries: ${lower_bound:,.2f} to ${upper_bound:,.2f}")

outliers = (df['salary_in_usd'] < lower_bound) | (df['salary_in_usd'] > upper_bound)
df_cleaned = df[~outliers].copy()

print(f"\nRemoved {outliers.sum():,} outliers ({outliers.sum()/len(df)*100:.2f}%)")
print(f"Salary range after cleaning: ${df_cleaned['salary_in_usd'].min():,.0f} - ${df_cleaned['salary_in_usd'].max():,.0f}")
print(f"Final dataset: {len(df_cleaned):,} rows")

Q1 (25th percentile): $112,700.00
Q3 (75th percentile): $200,740.00
IQR: $88,040.00
Outlier boundaries: $-19,360.00 to $332,800.00

Removed 3,418 outliers (2.54%)
Salary range after cleaning: $16,000 - $332,600
Final dataset: 131,407 rows


## Selecting Relevant Columns

We will now make sure to select the relevant columns we are going to use for the project

In [10]:
columns_to_keep = [
    'work_year',
    'experience_level',
    'job_title',
    'role_type',
    'salary_in_usd',
    'remote_ratio',
    'remote_work',
    'company_size'
]

df_final = df_cleaned[columns_to_keep].copy()

print(f"Final dataset shape: {df_final.shape[0]:,} rows × {df_final.shape[1]} columns")
df_final.head()

Final dataset shape: 131,407 rows × 8 columns


,work_year,experience_level,job_title,role_type,salary_in_usd,remote_ratio,remote_work,company_size
1,2025,Executive,Head of Data,Generalist,232344,0,0,M
2,2025,Senior,Data Scientist,Generalist,145400,0,0,M
3,2025,Senior,Data Scientist,Generalist,81600,0,0,M
4,2025,Mid,Engineer,Generalist,160000,100,1,M
5,2025,Mid,Engineer,Generalist,140000,100,1,M


## Export Cleaned Data

Finally, we will export the cleaned data

In [11]:
df_final.to_csv('cleaned_salaries.csv', index=False)

print(f"✓ Exported: cleaned_salaries.csv")
print(f"  {len(df_final):,} rows × {len(df_final.columns)} columns")
print(f"  Size: {df_final.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")

✓ Exported: cleaned_salaries.csv
  131,407 rows × 8 columns
  Size: 32.9 MB
